# Day 16

## Part 1

- Oh no... oh volca-no!
- 30 minutes before things get spicy
- flow rate is in pressure per minute (this bothers me on so many levels!)s
- it takes 1 minute to open a valve and 1 minute to follow a tunnel
- all valves begin closed
- total pressure releaved by a valve = time remaining * flow rate

`What is the most pressure you can release?`

In [77]:
from copy import deepcopy
from dataclasses import dataclass
from utils import parse_from_file, ParseConfig

def strip_plural(string: str) -> str:
    """
    required to account for usage of valve/valves
    """
    return string.strip('s ')

parser = ParseConfig('\n', ParseConfig('; ', [
    ParseConfig(' ', [
        None,  # Valve
        str,
        None, None,  # has flow
        ParseConfig('=', [None, int])
    ]),
    ParseConfig('valve', [None, ParseConfig(', ', strip_plural)])
]))

valves_and_caves = parse_from_file(
    'day_16.txt', parser, unnest_single_items=False)

print(valves_and_caves[0])
print(valves_and_caves[39])

[['ED', [0]], [['PS', 'AW']]]
[['RM', [18]], [['KJ']]]


In [78]:
# let's pop the input info into a nice structure
@dataclass
class Valve:
    name: str
    flow: int
    tunnels: list[str]|dict[str:int]

valves = {}
for (name, (flow, *_)), (tunnels, *_) in valves_and_caves:
    valves.update({name: Valve(name, flow, tunnels)})

print([contents for contents in list(valves.items())[:2]])

[('ED', Valve(name='ED', flow=0, tunnels=['PS', 'AW'])), ('SI', Valve(name='SI', flow=0, tunnels=['AA', 'HX']))]


In [79]:
# first we need a way to calculate the score of a given route
def calculate_score(route: tuple[str|None], route_length: int) -> int:
    """
    returns the score of a route

    in a route, strings represent moving between tunnels and None
    represents turning on a tap
    """
    last_stop = route[0]
    score = 0
    # the first item in a route is the starting point so don't count it
    for time_elapsed, event in enumerate(route[1:route_length], 1):
        if event is None:
            time_remaining = route_length - time_elapsed
            score += time_remaining * valves[last_stop].flow
        else:
            last_stop = event
    return score

route = ('AA', ) * 27 + ('OW', None, 'AA', 'AA')
print(calculate_score(route, 30))
# travel to AA: +0
# ...
# travel to OW: +0
# activate OW: +0
# travel to AA: +0 (+25)
# travel to AA: +0 (+25)
# so the total should be 50

50


In [80]:
# before we try all possible routes we can prune our net a little

# lots of nets have flow 0 so let's remove them and give each tunnel a weight
# to travel to all the tunnels with flow > 0

def get_paths(route: tuple[str]) -> dict[str:int]:
    """
    returns the travel cost to all non-zero flow valves from the valve passed
    """
    valve = valves[route[-1]]
    if valve.flow > 0 and len(route) > 1:
        return {valve.name: len(route) - 1}
    # get all routes to non zero flow nodes from all tunnels form this one
    shortest_routes = {}
    for option in valve.tunnels:
        # if we've already been there then don't bother checking
        # travel weights are all 1 so if it's in list it's quicker that way
        if option in route:
            continue
        temp = get_paths(route + (option, ))
        # update our best estimates of distance
        for desination, cost in temp.items():
            # replace an existing cost if we find a shorter route
            if desination in shortest_routes:
                if cost < shortest_routes[desination]:
                    shortest_routes[desination] = cost
            else:
                shortest_routes.update({desination: cost})
    return shortest_routes

valves_to_test = ('AA', 'HH')
for valve in valves_to_test:
    print(valve, get_paths((valve, )))

AA {'PB': 2, 'HX': 2, 'GJ': 3, 'AY': 3, 'PH': 2}
HH {'SV': 3, 'FY': 3, 'HX': 2}


In [81]:
# lets reduce the whole set now
reduced_valves = {}
for name, valve in valves.items():
    if valve.flow == 0 and not name == 'AA':
        continue
    new_valve = deepcopy(valve)
    new_valve.tunnels = get_paths((name, ))
    reduced_valves.update({name: new_valve})
    print(new_valve)

Valve(name='LX', flow=22, tunnels={'AW': 2, 'IN': 3})
Valve(name='PB', flow=4, tunnels={'BE': 3, 'HX': 2, 'AY': 2, 'GJ': 3, 'PH': 4})
Valve(name='PH', flow=11, tunnels={'GJ': 2, 'AW': 2, 'PB': 4, 'HX': 4, 'AY': 2})
Valve(name='SV', flow=24, tunnels={'QR': 2, 'HH': 3})
Valve(name='BE', flow=5, tunnels={'HX': 2, 'PB': 3, 'QR': 3, 'OW': 3})
Valve(name='AW', flow=21, tunnels={'PH': 2, 'LX': 2, 'GJ': 3, 'AY': 3})
Valve(name='FY', flow=17, tunnels={'HH': 3, 'RM': 2})
Valve(name='HH', flow=12, tunnels={'SV': 3, 'FY': 3, 'HX': 2})
Valve(name='AA', flow=0, tunnels={'PB': 2, 'HX': 2, 'GJ': 3, 'AY': 3, 'PH': 2})
Valve(name='HX', flow=14, tunnels={'QR': 3, 'HH': 2, 'PB': 2, 'GJ': 5, 'AY': 5, 'PH': 4, 'BE': 2})
Valve(name='GJ', flow=3, tunnels={'PB': 3, 'AW': 3, 'PH': 2, 'AY': 3, 'HX': 5})
Valve(name='RM', flow=18, tunnels={'FY': 2})
Valve(name='QR', flow=20, tunnels={'SV': 2, 'HX': 3, 'BE': 3, 'OW': 2})
Valve(name='AY', flow=6, tunnels={'PB': 2, 'AW': 3, 'HX': 5, 'GJ': 3, 'PH': 2})
Valve(name='OW'

In [82]:
# we need a check for which valves have been activated given a route
def is_activated(valve: str, route: tuple[str|None]):
    """
    returns true if route contains (..., valve, None,...)
    """
    for step1, step2 in zip(route[:-1], route[1:]):
        if step1 == valve and step2 is None:
            return True
    return False

In [83]:
# take a copy so we can re-run just this cell if need be
reduced_valves_copy = deepcopy(reduced_valves)

def get_scores(sub_route: tuple[str|None], total_time_left: int) -> dict:
    """
    returns a dictionary of the scores from completing the sub-route passed
    """
    # exit condition
    if len(sub_route) > total_time_left:
        return {sub_route: calculate_score(sub_route, total_time_left)}

    if sub_route[-1] is None:
        last_stop = sub_route[-2]
    else:
        last_stop = sub_route[-1]
    try:
        # figure out our options
        options = reduced_valves_copy[last_stop].tunnels
    except KeyError as e:
        print(sub_route)
        for _, valve in reduced_valves_copy.items():
            print(valve)
        raise e

    scores = {}
    # if this valve has not yet been acivated, activate it
    if not is_activated(last_stop, sub_route) and last_stop != 'AA':
        scores.update(get_scores(sub_route + (None, ), total_time_left))
    # else get scores from travelling to a new valve
    # it appears to be always better to activate a node than travel
    # hence let's not even bother checking routes that don't do this
    else:
        for option, cost in options.items():
            scores.update(
                get_scores(sub_route + (option, ) * cost, total_time_left))

    return scores


scores = get_scores(('AA',), 30)

In [84]:
best_route, best_score = None, None
for route, score in scores.items():
    if best_route is None:
        best_route, best_score = route, score
    elif score > best_score:
        best_route, best_score = route, score
print(best_route)
print(f'the most pressure we can relieve is {best_score}!')

('AA', 'PH', 'PH', None, 'AW', 'AW', None, 'LX', 'LX', None, 'IN', 'IN', 'IN', None, 'OW', 'OW', None, 'QR', 'QR', None, 'SV', 'SV', None, 'HH', 'HH', 'HH', None, 'HX', 'HX', None, 'QR', 'QR', 'QR')
the most pressure we can relieve is 2359!


## Part 2

- I can teach an elephant to help for the low low cost of only `4 minutes`
- An elephant can go it's own route and turn on valves as well

`With you and an elephant working together for 26 minutes, what is the most pressure you could release?`

In [85]:
# it is presumably still the most beneficial to always open a valve before
# moving on

# additionally it's probably also to always separate ways to the elephant to
# divide and conquer 

In [86]:
# with both of us working it may be useful to check if all valves have been
# opened
def all_valves_open(me: tuple[str|None], elephant: tuple[str|None]) -> bool:
    """
    returns true if between both of us all non-zero flow valves have been
    opened
    """
    for name, valve in reduced_valves.items():
        if not (is_activated(name, me) or is_activated(name, elephant)):
            return False
    return True

In [87]:

# with that in mind we need a modified score finding function
def get_dual_scores(
    me: tuple[str|None], elephant: tuple[str|None], total_time_left: int
) -> dict[tuple|int]:
    """
    returns a dictionary of scores of the combined routes possible following
    on from what we've already done.
    """
    # exit condition
    if len(me) > total_time_left or all_valves_open(me, elephant):
        return [(me, elephant)]

    if me[-1] is None:
        my_last_stop = me[-2]
    else:
        my_last_stop = me[-1]

    my_options = reduced_valves_copy[my_last_stop].tunnels

    if elephant[-1] is None:
        elephant_last = elephant[-2]
    else:
        elephant_last = elephant[-1]

    my_options = reduced_valves_copy[my_last_stop].tunnels
    elephant_opts = reduced_valves_copy[elephant_last].tunnels

    scores = {}
    my_stop_is_active = (
        is_activated(my_last_stop, me) or is_activated(my_last_stop, elephant)
    )
    elephant_stop_active = (
        is_activated(elephant_last, me) or is_activated(elephant_last, elephant)
    )
    # choose my next move
    if not my_stop_is_active and my_last_stop != 'AA':
        my_next_moves = [me + (None, )]
    else:
        my_next_moves = [me + (option, ) * cost for option, cost in my_options.items()]
    # choose the elephant's next move
    if not elephant_stop_active and elephant_last != my_last_stop:
        elephant_next_moves = {my_next: [elephant + (None, )] for my_next in my_next_moves}
    else:
        elephant_next_moves = {}
        for my_option in my_next_moves:
            temp = []
            for option, cost in elephant_opts.items():
                elephant_option = elephant + (option, ) * cost
                if elephant_option != my_option:
                    temp.append(elephant_option)
            elephant_next_moves.update({my_option: temp})

    scores = []
    for my_next_move in my_next_moves:
        # print(my_next_move)
        # print(elephant_next_moves)
        for elephant_next_move in elephant_next_moves[my_next_move]:
            scores.extend(get_dual_scores(
                my_next_move, elephant_next_move, total_time_left))

    return scores

dual_scores = get_dual_scores(('AA', ), ('AA', ), 12)

In [88]:
best_route, best_score = None, None
for route in dual_scores:
    m, e = route
    score = calculate_score(m, 26) + calculate_score(e, 26)
    if best_route is None:
        best_route, best_score = route, score
        continue
    if score > best_score:
        best_route, best_score = route, score
print(best_route)
print(f'the most pressure we can relieve is {best_score}!')

(('AA', 'PH', 'PH', None, 'AW', 'AW', None, 'LX', 'LX', None, 'AW', 'AW', 'PH', 'PH'), ('AA', 'HX', 'HX', None, 'QR', 'QR', 'QR', None, 'OW', 'OW', None, 'IN', 'IN', None))
the most pressure we can relieve is 2357!
